**British Airways Revenue Forecasting 1** <br>
Oxford Saïd Business School <br>
Analysis 1: Data Cleaning <br>
<br>
*Jake Schneider* <br>
MBA Candidate <br>
November 2020

---

**Load Libraries**

In [1]:
#Import libraries
import sys
import pandas as pd
from datetime import date, datetime, time, timedelta
import pendulum
import json
import requests
import numpy as np
from numpy import NaN
import math

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='ticks', context='talk')

from matplotlib.offsetbox import AnchoredText
from sklearn import linear_model
from sklearn.metrics import r2_score
import statsmodels.api as sm
#import fbprophet

import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import datapane as dp

import warnings
import itertools

----

**Load Data**

In [2]:
# Load Data from World Bank World Development Indicators 
# Used wbdata API

wdi_df = pd.read_csv('../2_Inputs/Country DF/country_df.csv')

In [3]:
# Load Data from World Bank Population estimates and projections
# Data available from: https://databank.worldbank.org/source/population-estimates-and-projections#

wb_df = pd.read_csv('../2_Inputs/World Bank 2/World Bank Population Projections Final.csv')

In [4]:
# Load Data from IMF World Economic Outlook database: October 2020
# Data available from: https://www.imf.org/en/Publications/WEO/weo-database/2020/October/download-entire-database

imf_df = pd.read_csv('../2_Inputs/IMF/WEOOct2020all.csv')

---

**Data Transformations**

*WB DF*

In [5]:
# View WB Data

#print(wb_df.dtypes)
wb_df.head()

,Country Name,Country Code,Series Name,Series Code,1980,1981,1982,1983,1984,1985,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Afghanistan,AFG,Population growth (annual %),SP.POP.GROW,-4.075460e-01,-1.393544e+00,-2.219656e+00,-2.712948e+00,-2.695480e+00,-2.204368e+00,...,2.778317e+00,2.548347e+00,2.384761e+00,2.311817e+00,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,1.335651e+07,1.317167e+07,1.288253e+07,1.253773e+07,1.220429e+07,1.193821e+07,...,3.538313e+07,3.629640e+07,3.717239e+07,3.804175e+07,38928000.0,39835000.0,40754000.0,41681000.0,42609000.0,43532000.0
2,Albania,ALB,Population growth (annual %),SP.POP.GROW,2.047964e+00,2.002974e+00,2.113272e+00,2.120885e+00,2.103937e+00,2.055995e+00,...,-1.598804e-01,-9.197229e-02,-2.467320e-01,-4.260074e-01,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,2.671997e+06,2.726056e+06,2.784278e+06,2.843960e+06,2.904429e+06,2.964762e+06,...,2.876101e+06,2.873457e+06,2.866376e+06,2.854191e+06,2850000.0,2844000.0,2837000.0,2828000.0,2820000.0,2811000.0
4,Algeria,DZA,Population growth (annual %),SP.POP.GROW,3.030905e+00,3.087048e+00,3.121627e+00,3.123712e+00,3.087580e+00,3.022861e+00,...,2.051355e+00,2.044953e+00,2.007374e+00,1.933950e+00,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Create years list

years_list = list(range(1980, 2026))
#print(years_list)
years_list2 = list(map(str,  years_list))
print(years_list2)

['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']


In [7]:
# Reshape Wide to Long

wb_df = wb_df.reset_index()
wb_df = pd.melt(wb_df, id_vars=['Country Name', 'Country Code', 'Series Name'], 
                value_vars=years_list2)
wb_df

,Country Name,Country Code,Series Name,variable,value
0,Afghanistan,AFG,Population growth (annual %),1980,-4.075460e-01
1,Afghanistan,AFG,"Population, total",1980,1.335651e+07
2,Albania,ALB,Population growth (annual %),1980,2.047964e+00
3,Albania,ALB,"Population, total",1980,2.671997e+06
4,Algeria,DZA,Population growth (annual %),1980,3.030905e+00
...,...,...,...,...,...
19959,"Yemen, Rep.",YEM,"Population, total",2025,3.314000e+07
19960,Zambia,ZMB,Population growth (annual %),2025,NaN
19961,Zambia,ZMB,"Population, total",2025,2.119700e+07
19962,Zimbabwe,ZWE,Population growth (annual %),2025,NaN


In [8]:
# Rename variable to Year

wb_df = wb_df.rename(columns={"variable": "Year"})
wb_df.head()

,Country Name,Country Code,Series Name,Year,value
0,Afghanistan,AFG,Population growth (annual %),1980,-4.075460e-01
1,Afghanistan,AFG,"Population, total",1980,1.335651e+07
2,Albania,ALB,Population growth (annual %),1980,2.047964e+00
3,Albania,ALB,"Population, total",1980,2.671997e+06
4,Algeria,DZA,Population growth (annual %),1980,3.030905e+00


In [9]:
# Unstack

#wb_df = wb_df.set_index(['Country Name', 'Country Code', 'Year'], append=True)
#wb_df.head()

wb_df = wb_df.set_index(['Country Name', 'Country Code', 'Year', 'Series Name']).unstack('Series Name')
#print(wb_df.dtypes)
wb_df                                                                                       

value                  
Series Name                    Population growth (annual %) Population, total
Country Name Country Code Year                                               
Afghanistan  AFG          1980                    -0.407546        13356511.0
                          1981                    -1.393544        13171673.0
                          1982                    -2.219656        12882528.0
                          1983                    -2.712948        12537730.0
                          1984                    -2.695480        12204292.0
...                                                     ...               ...
Zimbabwe     ZWE          2021                          NaN        15092000.0
                          2022                          NaN        15331000.0
                          2023                          NaN        15581000.0
                          2024                          NaN        15841000.0
                          2025                          NaN        16110000.0

[9982 rows x 2 columns]

In [10]:
# Drop Top Level

wb_df.columns = wb_df.columns.droplevel()
#print(wb_df.dtypes)
wb_df

Series Name                     Population growth (annual %)  \
Country Name Country Code Year                                 
Afghanistan  AFG          1980                     -0.407546   
                          1981                     -1.393544   
                          1982                     -2.219656   
                          1983                     -2.712948   
                          1984                     -2.695480   
...                                                      ...   
Zimbabwe     ZWE          2021                           NaN   
                          2022                           NaN   
                          2023                           NaN   
                          2024                           NaN   
                          2025                           NaN   

Series Name                     Population, total  
Country Name Country Code Year                     
Afghanistan  AFG          1980         13356511.0  
                          1981         13171673.0  
                          1982         12882528.0  
                          1983         12537730.0  
                          1984         12204292.0  
...                                           ...  
Zimbabwe     ZWE          2021         15092000.0  
                          2022         15331000.0  
                          2023         15581000.0  
                          2024         15841000.0  
                          2025         16110000.0  

[9982 rows x 2 columns]

In [11]:
# Output Data

wb_df = wb_df.to_csv('../3_Outputs/Data/world_bank_data_clean.csv', index = True)

*IMF DF*

In [12]:
#View IMF Data

imf_df

,WEO Country Code,ISO,WEO Subject Code,Country,Subject Descriptor,Subject Descriptor 2,Subject Notes,Units,Scale,Country/Series-specific Notes,...,2017,2018,2019,2020,2021,2022,2023,2024,2025,Estimates Start After
0,512,AFG,NGDP_R,Afghanistan,"Gross domestic product, constant prices, Natio...","Gross domestic product, constant prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office Latest actu...,...,"1,255.29","1,270.22","1,319.90","1,253.91","1,304.06","1,363.06","1,424.40","1,481.50","1,540.94",2019.0
1,512,AFG,NGDP_RPCH,Afghanistan,"Gross domestic product, constant prices, Perce...","Gross domestic product, constant prices",Annual percentages of constant price GDP are y...,Percent change,NaN,"See notes for: Gross domestic product, consta...",...,2.647,1.189,3.912,-5,4,4.524,4.5,4.009,4.012,2019.0
2,512,AFG,NGDP,Afghanistan,"Gross domestic product, current prices, Nation...","Gross domestic product, current prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office Latest actu...,...,"1,285.46","1,327.69","1,469.60","1,465.92","1,597.74","1,741.83","1,893.02","2,047.67","2,215.01",2019.0
3,512,AFG,NGDPD,Afghanistan,"Gross domestic product, current prices, U.S. d...","Gross domestic product, current prices",Values are based upon GDP in national currency...,U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",...,18.91,18.401,18.876,19.006,19.692,20.829,22.022,23.169,24.372,2019.0
4,512,AFG,PPPGDP,Afghanistan,"Gross domestic product, current prices, Purcha...","Gross domestic product, current prices",These data form the basis for the country weig...,Purchasing power parity; international dollars,Billions,"See notes for: Gross domestic product, curren...",...,74.712,77.416,81.88,78.884,83.852,89.205,94.908,100.6,106.685,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8770,698,ZWE,GGXWDG,Zimbabwe,"General government gross debt, National curren...",General government gross debt,Gross debt consists of all liabilities that re...,National currency,Billions,Source: Ministry of Finance or Treasury Latest...,...,14.505,15.859,17.224,18.512,19.73,20.862,21.46,22.034,22.644,2018.0
8771,698,ZWE,GGXWDG_NGDP,Zimbabwe,"General government gross debt, Percent of GDP,",General government gross debt,Gross debt consists of all liabilities that re...,Percent of GDP,NaN,See notes for: General government gross debt ...,...,52.866,37.343,10.806,2.358,2.226,2.202,2.148,2.06,1.975,2018.0
8772,698,ZWE,NGDP_FY,Zimbabwe,Gross domestic product corresponding to fiscal...,Gross domestic product corresponding to fiscal...,Gross domestic product corresponding to fiscal...,National currency,Billions,Source: Ministry of Finance or Treasury Latest...,...,27.438,42.468,159.391,785.156,886.282,947.567,999.093,"1,069.72","1,146.46",2018.0
8773,698,ZWE,BCA,Zimbabwe,"Current account balance, U.S. dollars, Billions",Current account balance,Current account is all transactions other than...,U.S. dollars,Billions,Source: Reserve Bank of Zimbabwe and Ministry ...,...,-0.284,-1.229,0.208,-0.505,-0.16,-0.288,-0.589,-0.603,-0.54,2018.0


In [13]:
# Reshape Wide to Long

imf_df = imf_df.reset_index()
imf_df = pd.melt(imf_df, id_vars=['Country', 'ISO', 'Subject Descriptor'], 
                 value_vars=years_list2)
imf_df

,Country,ISO,Subject Descriptor,variable,value
0,Afghanistan,AFG,"Gross domestic product, constant prices, Natio...",1980,NaN
1,Afghanistan,AFG,"Gross domestic product, constant prices, Perce...",1980,NaN
2,Afghanistan,AFG,"Gross domestic product, current prices, Nation...",1980,NaN
3,Afghanistan,AFG,"Gross domestic product, current prices, U.S. d...",1980,NaN
4,Afghanistan,AFG,"Gross domestic product, current prices, Purcha...",1980,NaN
...,...,...,...,...,...
403645,Zimbabwe,ZWE,"General government gross debt, National curren...",2025,22.644
403646,Zimbabwe,ZWE,"General government gross debt, Percent of GDP,",2025,1.975
403647,Zimbabwe,ZWE,Gross domestic product corresponding to fiscal...,2025,"1,146.46"
403648,Zimbabwe,ZWE,"Current account balance, U.S. dollars, Billions",2025,-0.54


In [14]:
# Rename variable to Year

imf_df = imf_df.rename(columns={"variable": "Year"})
imf_df.head()

,Country,ISO,Subject Descriptor,Year,value
0,Afghanistan,AFG,"Gross domestic product, constant prices, Natio...",1980,NaN
1,Afghanistan,AFG,"Gross domestic product, constant prices, Perce...",1980,NaN
2,Afghanistan,AFG,"Gross domestic product, current prices, Nation...",1980,NaN
3,Afghanistan,AFG,"Gross domestic product, current prices, U.S. d...",1980,NaN
4,Afghanistan,AFG,"Gross domestic product, current prices, Purcha...",1980,NaN


In [15]:
# Unstack

#wb_df = wb_df.set_index(['Country Name', 'Country Code', 'Year'], append=True)
#wb_df.head()

imf_df = imf_df.set_index(['Country', 'ISO', 'Year', 'Subject Descriptor']).unstack('Subject Descriptor')
imf_df    

value  \
Subject Descriptor   Current account balance, Percent of GDP,    
Country     ISO Year                                             
Afghanistan AFG 1980                                       NaN   
                1981                                       NaN   
                1982                                       NaN   
                1983                                       NaN   
                1984                                       NaN   
...                                                        ...   
Zimbabwe    ZWE 2021                                    -2.005   
                2022                                    -3.416   
                2023                                    -6.735   
                2024                                    -6.534   
                2025                                    -5.537   

                                                                      \
Subject Descriptor   Current account balance, U.S. dollars, Billions   
Country     ISO Year                                                   
Afghanistan AFG 1980                                             NaN   
                1981                                             NaN   
                1982                                             NaN   
                1983                                             NaN   
                1984                                             NaN   
...                                                              ...   
Zimbabwe    ZWE 2021                                           -0.16   
                2022                                          -0.288   
                2023                                          -0.589   
                2024                                          -0.603   
                2025                                           -0.54   

                                                    \
Subject Descriptor   Employment, Persons, Millions   
Country     ISO Year                                 
Afghanistan AFG 1980                           NaN   
                1981                           NaN   
                1982                           NaN   
                1983                           NaN   
                1984                           NaN   
...                                            ...   
Zimbabwe    ZWE 2021                           NaN   
                2022                           NaN   
                2023                           NaN   
                2024                           NaN   
                2025                           NaN   

                                                                                 \
Subject Descriptor   General government gross debt, National currency, Billions   
Country     ISO Year                                                              
Afghanistan AFG 1980                                                NaN           
                1981                                                NaN           
                1982                                                NaN           
                1983                                                NaN           
                1984                                                NaN           
...                                                                 ...           
Zimbabwe    ZWE 2021                                              19.73           
                2022                                             20.862           
                2023                                              21.46           
                2024                                             22.034           
                2025                                             22.644           

                                                                      \
Subject Descriptor   General government gross debt, Percent of GDP,    
Country     ISO Year                                       

In [16]:
# Drop Top Level

imf_df.columns = imf_df.columns.droplevel()
imf_df

Subject Descriptor   Current account balance, Percent of GDP,   \
Country     ISO Year                                             
Afghanistan AFG 1980                                       NaN   
                1981                                       NaN   
                1982                                       NaN   
                1983                                       NaN   
                1984                                       NaN   
...                                                        ...   
Zimbabwe    ZWE 2021                                    -2.005   
                2022                                    -3.416   
                2023                                    -6.735   
                2024                                    -6.534   
                2025                                    -5.537   

Subject Descriptor   Current account balance, U.S. dollars, Billions  \
Country     ISO Year                                                   
Afghanistan AFG 1980                                             NaN   
                1981                                             NaN   
                1982                                             NaN   
                1983                                             NaN   
                1984                                             NaN   
...                                                              ...   
Zimbabwe    ZWE 2021                                           -0.16   
                2022                                          -0.288   
                2023                                          -0.589   
                2024                                          -0.603   
                2025                                           -0.54   

Subject Descriptor   Employment, Persons, Millions  \
Country     ISO Year                                 
Afghanistan AFG 1980                           NaN   
                1981                           NaN   
                1982                           NaN   
                1983                           NaN   
                1984                           NaN   
...                                            ...   
Zimbabwe    ZWE 2021                           NaN   
                2022                           NaN   
                2023                           NaN   
                2024                           NaN   
                2025                           NaN   

Subject Descriptor   General government gross debt, National currency, Billions  \
Country     ISO Year                                                              
Afghanistan AFG 1980                                                NaN           
                1981                                                NaN           
                1982                                                NaN           
                1983                                                NaN           
                1984                                                NaN           
...                                                                 ...           
Zimbabwe    ZWE 2021                                              19.73           
                2022                                             20.862           
                2023                                              21.46           
                2024                                             22.034           
                2025                                             22.644           

Subject Descriptor   General government gross debt, Percent of GDP,   \
Country     ISO Year                                                   
Afghanistan AFG 1980                                             NaN   
                1981                                             NaN   
                1982                                             NaN   
                1983                                         

In [17]:
# Output Data

imf_df = imf_df.to_csv('../3_Outputs/Data/imf_clean.csv', index = True)

---

**Merge Datasets Together**

In [18]:
# Load Final WB Dataframe

wb_final_df = pd.read_csv('../3_Outputs/Data/world_bank_data_clean.csv')
wb_final_df

,Country Name,Country Code,Year,Population growth (annual %),"Population, total"
0,Afghanistan,AFG,1980,-0.407546,13356511.0
1,Afghanistan,AFG,1981,-1.393544,13171673.0
2,Afghanistan,AFG,1982,-2.219656,12882528.0
3,Afghanistan,AFG,1983,-2.712948,12537730.0
4,Afghanistan,AFG,1984,-2.695480,12204292.0
...,...,...,...,...,...
9977,Zimbabwe,ZWE,2021,NaN,15092000.0
9978,Zimbabwe,ZWE,2022,NaN,15331000.0
9979,Zimbabwe,ZWE,2023,NaN,15581000.0
9980,Zimbabwe,ZWE,2024,NaN,15841000.0


In [19]:
# Load Final IMF Dataframe

imf_final_df = pd.read_csv('../3_Outputs/Data/imf_clean.csv')
imf_final_df

,Country,ISO,Year,"Current account balance, Percent of GDP,","Current account balance, U.S. dollars, Billions","Employment, Persons, Millions","General government gross debt, National currency, Billions","General government gross debt, Percent of GDP,","General government net debt, National currency, Billions","General government net debt, Percent of GDP,",...,"Inflation, end of period consumer prices, Percent change,","Output gap in percent of potential GDP, Percent of potential GDP,","Population, Persons, Millions","Six-month London interbank offered rate (LIBOR), Percent,","Total investment, Percent of GDP,","Unemployment rate, Percent of total labor force,","Volume of Imports of goods, Percent change,","Volume of exports of goods and services, Percent change,","Volume of exports of goods, Percent change,","Volume of imports of goods and services, Percent change,"
0,Afghanistan,AFG,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8965,Zimbabwe,ZWE,2021,-2.005,-0.16,NaN,19.73,2.226,NaN,NaN,...,3,NaN,15.492,NaN,NaN,NaN,29.599,NaN,6.111,NaN
8966,Zimbabwe,ZWE,2022,-3.416,-0.288,NaN,20.862,2.202,NaN,NaN,...,3,NaN,15.817,NaN,NaN,NaN,30.919,NaN,7.548,NaN
8967,Zimbabwe,ZWE,2023,-6.735,-0.589,NaN,21.46,2.148,NaN,NaN,...,3,NaN,16.164,NaN,NaN,NaN,32.17,NaN,5.947,NaN
8968,Zimbabwe,ZWE,2024,-6.534,-0.603,NaN,22.034,2.060,NaN,NaN,...,3,NaN,16.481,NaN,NaN,NaN,33.808,NaN,5.831,NaN


In [20]:
# Merge Country_DF and WB_DF

merge_df1 = pd.merge(wdi_df, wb_final_df, how = 'outer', left_on = ['id'], right_on = ['Country Code'])
merge_df1.head()

,id,iso2Code,country,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude,Country Name,Country Code,Year,Population growth (annual %),"Population, total"
0,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,Afghanistan,AFG,1980.0,-0.407546,13356511.0
1,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,Afghanistan,AFG,1981.0,-1.393544,13171673.0
2,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,Afghanistan,AFG,1982.0,-2.219656,12882528.0
3,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,Afghanistan,AFG,1983.0,-2.712948,12537730.0
4,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,Afghanistan,AFG,1984.0,-2.695480,12204292.0


In [21]:
# Merge Merge1 and IMF_DF

df = pd.merge(merge_df1, imf_final_df, how = 'left', left_on = ['id', 'Year'], right_on = ['ISO', 'Year'])
df.head()

,id,iso2Code,country,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude,...,"Inflation, end of period consumer prices, Percent change,","Output gap in percent of potential GDP, Percent of potential GDP,","Population, Persons, Millions","Six-month London interbank offered rate (LIBOR), Percent,","Total investment, Percent of GDP,","Unemployment rate, Percent of total labor force,","Volume of Imports of goods, Percent change,","Volume of exports of goods and services, Percent change,","Volume of exports of goods, Percent change,","Volume of imports of goods and services, Percent change,"
0,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Output DF Final Data

df = df.to_csv('../3_Outputs/Data/ba_df.csv', index = False)